In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from icevision.imports import *
from icevision.models.multitask.ultralytics.yolov5 import *
from icevision.data.data_splitter import *
from icevision.visualize import *
from icevision.metrics import *

import icedata.datasets.exdark_trimmed as exdark

In [ ]:
IMG_SIZE=512
data_dir = exdark.load_data()
parser = exdark.parser(data_dir)

train_records, valid_records = parser.parse(data_splitter=RandomSplitter([0.8, 0.2]))
train_tfms = tfms.A.Adapter(
    [
        *tfms.A.aug_tfms(size=IMG_SIZE, lightning=None),
        tfms.A.Normalize(),
    ]
)
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(IMG_SIZE), tfms.A.Normalize()])

train_ds = Dataset(train_records, tfm=train_tfms)
valid_ds = Dataset(valid_records, tfm=valid_tfms)

In [ ]:
dl_train = train_dl(train_ds, batch_size=32)
dl_valid = valid_dl(valid_ds, batch_size=64)

In [ ]:
hybrid_model = model(
    backbone=backbones.large(pretrained=True),
    num_detection_classes=len(parser.CLASS_MAPS['detection']),
    classifier_configs={
        name: ClassifierConfig(out_classes=len(cm))
        for name, cm in parser.CLASS_MAPS.items() if not name=="detection"
    },
    img_size=IMG_SIZE,
)


In [ ]:
from torch import optim
import pytorch_lightning as pl

class LightModel(lightning.HybridYOLOV5LightningAdapter):
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-4)

pl_model = LightModel(
    model=hybrid_model,
    metrics=[COCOMetric(metric_type=COCOMetricType.bbox)],
)

In [ ]:
trainer = pl.Trainer(max_epochs=20, gpus=[0])
trainer

In [ ]:
trainer.fit(pl_model, dl_train, dl_valid)

---

In [ ]:
from icevision.models.multitask.ultralytics.yolov5.prediction import *

In [ ]:
valid_ds = Dataset(valid_records[:20], tfm=valid_tfms)

In [ ]:
preds = predict(
    model=pl_model.model,
    dataset=valid_ds,
    detection_threshold=0.4,
    keep_images=True,
)

In [ ]:
draw_sample = partial(draw_sample, denormalize_fn=denormalize_imagenet, return_as_pil_img=True)

In [ ]:
import fastcore.all as fastcore
import PIL
import PIL.Image

@fastcore.patch
def __or__(self: PIL.Image.Image, other: PIL.Image.Image):
    "Horizontally stack two PIL Images"
    assert isinstance(other, PIL.Image.Image)
    widths, heights = zip(*(i.size for i in [self, other]))

    new_img = PIL.Image.new("RGB", (sum(widths), max(heights)))
    x_offset = 0
    for img in [self, other]:
        new_img.paste(img, (x_offset, 0))
        x_offset += img.size[0]
    return new_img

In [ ]:
pred = preds[19]
p, gt = pred.pred, pred.ground_truth

draw_sample(gt) | draw_sample(p)